In [ ]:
import pandas as pd
import os
from scipy.io import mmread
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

All sequencing data used in this study was obtained from the Gene Expression Omnibus repository under the accession code GSE219281.

In [ ]:
rna_folder_path = "/Users/elizabethchang/Library/CloudStorage/GoogleDrive-ec3055@columbia.edu/My Drive/final projects CBMFW4761 BINFG4002/data/c9als data/GSE219281_RAW/raw snRNA motor cortex"
atac_folder_path = "/Users/elizabethchang/Library/CloudStorage/GoogleDrive-ec3055@columbia.edu/My Drive/final projects CBMFW4761 BINFG4002/data/c9als data/GSE219281_RAW/raw snATAC motor cortex"

patients = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6']

cell_types = ['Astro', 'Exc']

rna_files = {}
atac_files = {}

for patient in patients:
    rna_files[patient] = ["matrix", "features", "barcodes"]

rna_metadata = pd.read_csv("snRNA metadata.csv")
rna_metadata = rna_metadata[rna_metadata["annotation_major_cell_type"].isin(cell_types)]

atac_metadata = pd.read_csv("snATAC metadata.csv")
atac_metadata = atac_metadata[atac_metadata["annotation_major_cell_type"].isin(cell_types)]

genes = list(pd.read_csv("TF_target_binary_matrix.csv")["gene"])

In [ ]:
for file in os.listdir(rna_folder_path):
    split_string = file.split("_")
    if len(split_string) > 3:
        file_type = split_string[-1]
        patient = split_string[3]
        file_path = os.path.join(rna_folder_path, file)

        if file_type == "features.tsv":
            rna_files[patient][1] = file_path
        elif file_type == "matrix.mtx":
            rna_files[patient][0] = file_path
        elif file_type == "barcodes.tsv":
            rna_files[patient][2] = file_path

for file in os.listdir(atac_folder_path):
    split_string = file.split("_")
    if len(split_string) > 3:
        cohort = split_string[2]
        patient = split_string[3]
        file_type = split_string[-1]
        file_path = os.path.join(atac_folder_path, file)
        if file_type == "fragments.tsv":
            atac_files[patient] = file_path


In [ ]:
window = 1000

for patient in tqdm(patients):
    atac_file = atac_files[patient]

    atac_df = pd.read_csv(atac_file, sep="\t", header = None)
    atac_df.columns = ["peak", "start", "end", "cell_barcode", "count"]
    atac_df = atac_df[~atac_df["peak"].isin(["chrX", "chrY"])]
    patient_atac_metadata = atac_metadata[atac_metadata["donor_id"] == patient][["cell_barcode","annotation_major_cell_type"]]
    atac_df = pd.merge(patient_atac_metadata, atac_df, on="cell_barcode")

    atac_df['start'] = (atac_df['start'] // window) * window
    atac_df['end'] = (atac_df['end'] // window) * window
    atac_df['end'] = atac_df.apply(lambda row: row['end'] + window if row['end'] == row['start'] else row['end'], axis=1)
    atac_df["peak"] = (atac_df["peak"] + '_' + atac_df["start"].astype(str)+ '_' + atac_df["end"].astype(str))
    atac_df.drop(columns=["start", "end"], inplace=True)

    features_file = rna_files[patient][1]
    matrix_file = rna_files[patient][0]
    barcodes_file = rna_files[patient][2]

    features = pd.read_csv(features_file, sep="\t", header=None)
    barcodes = pd.read_csv(barcodes_file, sep="\t", header=None)
    sc_data = mmread(matrix_file)
    csr_data = csr_matrix(sc_data).transpose()
    matrix_df = pd.DataFrame.sparse.from_spmatrix(csr_data)
    matrix_df.columns = features.iloc[:, 1].tolist()

    rna_df = pd.merge(barcodes, matrix_df, left_index=True,right_index=True)
    rna_df.columns.values[0] = "cell_barcode"

    patient_rna_metadata = rna_metadata[rna_metadata["donor_id"] == patient]
    rna_df = pd.merge(patient_rna_metadata[["cell_barcode", "annotation_major_cell_type"]], rna_df, on="cell_barcode" )

    for cell in cell_types:
        cell_atac_df = atac_df[atac_df["annotation_major_cell_type"] == cell].copy()
        top_peaks = cell_atac_df.groupby('peak')['count'].sum().nlargest(50000).index
        cell_atac_df = cell_atac_df[cell_atac_df['peak'].isin(top_peaks)]
        atac_barcode_mapping = {barcode: i+1 for i, barcode in enumerate(cell_atac_df['cell_barcode'].unique())}
        cell_atac_df['cell_barcode'] = cell_atac_df['cell_barcode'].map(atac_barcode_mapping)
        cell_atac_df['cell_id'] = cell + '_' + cell_atac_df['cell_barcode'].astype(str)
        cell_atac_df.drop(columns=['cell_barcode','annotation_major_cell_type'], inplace=True)

        atac_pivot = cell_atac_df.pivot_table(index='peak', columns='cell_id', values='count', fill_value=0)
        atac_pivot.reset_index(inplace=True)
        atac_pivot.to_csv(f"output/{patient}_{cell}_ATAC.csv", index=False)

        cell_rna_df = rna_df[rna_df["annotation_major_cell_type"] == cell].copy()
        rna_barcode_mapping = {barcode: i+1 for i, barcode in enumerate(cell_rna_df['cell_barcode'].unique())}
        cell_rna_df['cell_barcode'] = cell_rna_df['cell_barcode'].map(rna_barcode_mapping)
        cell_rna_df['cell_id'] = cell + '_' + cell_rna_df['cell_barcode'].astype(str)
        rna_transposed_df = cell_rna_df.set_index(cell_rna_df.columns[-1]).T.reset_index()
        rna_transposed_df.rename(columns={'index': 'gene'}, inplace=True)
        rna_transposed_df = rna_transposed_df[rna_transposed_df['gene'].isin(genes)]
        rna_transposed_df.to_csv(f"output/{patient}_{cell}_RNA.csv", index=False)

100%|██████████| 12/12 [46:11<00:00, 230.94s/it]


In [ ]:
for file in sorted(os.listdir("output")):
    df = pd.read_csv(f"output/{file}", encoding='unicode_escape')
    print(f"{file}: {df.shape}")

A1_Astro_ATAC.csv: (50000, 201)
A1_Astro_RNA.csv: (17197, 911)
A1_Exc_ATAC.csv: (50000, 198)
A1_Exc_RNA.csv: (17197, 533)
A2_Astro_ATAC.csv: (50000, 235)
A2_Astro_RNA.csv: (17197, 352)
A2_Exc_ATAC.csv: (50000, 1244)
A2_Exc_RNA.csv: (17197, 866)
A3_Astro_ATAC.csv: (50000, 246)
A3_Astro_RNA.csv: (17197, 474)
A3_Exc_ATAC.csv: (50000, 1014)
A3_Exc_RNA.csv: (17197, 1042)
A4_Astro_ATAC.csv: (50000, 625)
A4_Astro_RNA.csv: (17197, 86)
A4_Exc_ATAC.csv: (50000, 1136)
A4_Exc_RNA.csv: (17197, 78)
A5_Astro_ATAC.csv: (50000, 237)
A5_Astro_RNA.csv: (17197, 454)
A5_Exc_ATAC.csv: (50000, 446)
A5_Exc_RNA.csv: (17197, 556)
A6_Astro_ATAC.csv: (50000, 154)
A6_Astro_RNA.csv: (17197, 629)
A6_Exc_ATAC.csv: (50000, 505)
A6_Exc_RNA.csv: (17197, 504)
C1_Astro_ATAC.csv: (50000, 609)
C1_Astro_RNA.csv: (17197, 1020)
C1_Exc_ATAC.csv: (50000, 1169)
C1_Exc_RNA.csv: (17197, 386)
C2_Astro_ATAC.csv: (50000, 242)
C2_Astro_RNA.csv: (17197, 284)
C2_Exc_ATAC.csv: (50000, 771)
C2_Exc_RNA.csv: (17197, 206)
C3_Astro_ATAC.csv: (

In [ ]:
def subset(merged_df, t, cells = 250, rows = 10000):
    data = "peak" if t=="ATAC" else "gene"
    row_sums = merged_df.iloc[:, 1:].sum(axis=1)
    col_sums = merged_df.iloc[:, 1:].sum(axis=0)
    merged_df["sum"] = row_sums
    merged_df = merged_df.nlargest(columns="sum", n=rows).drop(columns="sum")
    columns_to_keep = [data] + list(col_sums.nlargest(cells).index)
    data_df = merged_df[columns_to_keep]
    data_df = pd.concat([merged_df[data],data_df.iloc[:, 1:].map(math.ceil)],axis=1)
    return data_df

In [ ]:
folder = "output rna and atac 50000"

control = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
als = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6']
cohorts = [control, als]
cohort_names = ["control", "als"]

astro_atac = {}
astro_rna = {}
exc_atac = {}
exc_rna = {}

for file in sorted(os.listdir(folder)):
    if file.endswith(".csv"):
        file_path = os.path.join(folder,file)
        temp = file[:-4].split("_")
        patient = temp[0]
        data = temp[2]
        cell = temp[1]

        if cell == "Astro":
            if data == "ATAC":
                astro_atac[patient] = file_path
            else:
                astro_rna[patient] = file_path
        else:
            if data == "ATAC":
                exc_atac[patient] = file_path
            else:
                exc_rna[patient] = file_path

dict_names = ['astro_atac', 'astro_rna', 'exc_atac', 'exc_rna']
files_list = [astro_atac, astro_rna, exc_atac, exc_rna]

for file_dict, dict_name in zip(files_list, dict_names):
    for cohort in cohorts:
        df_list = []
        for patient in cohort:
            df = pd.read_csv(file_dict[patient])
            df_list.append(df)
        merged_df = None
        suffix_number = 0
        for df in df_list:
            if merged_df is None:
                merged_df = df
            else:
                suffix_number += 1
                suffix = '_' + str(suffix_number)
                merged_df = pd.merge(merged_df, df, on="peak" if "atac" in dict_name else "gene", suffixes=('', suffix), how = "outer")
        c = "Control" if "C1" in cohort else "ALS"
        t = "ATAC" if "atac" in dict_name else "RNA"
        k = "Astro" if "astro" in dict_name else "Exc"
        file_name = f"merged all peaks subset/subset_{c}_{t}_{k}.csv"
        merged_df.fillna(0, inplace=True)
        data_df = subset(merged_df, t)
        data_df.to_csv(file_name, index=False)
        print(file_name, data_df.shape)

merged all peaks subset/subset_Control_ATAC_Astro.csv (10000, 251)
merged all peaks subset/subset_ALS_ATAC_Astro.csv (10000, 251)
merged all peaks subset/subset_Control_RNA_Astro.csv (10000, 251)
merged all peaks subset/subset_ALS_RNA_Astro.csv (10000, 251)
merged all peaks subset/subset_Control_ATAC_Exc.csv (10000, 251)
merged all peaks subset/subset_ALS_ATAC_Exc.csv (10000, 251)
merged all peaks subset/subset_Control_RNA_Exc.csv (10000, 251)
merged all peaks subset/subset_ALS_RNA_Exc.csv (10000, 251)


In [ ]:
for file in sorted(os.listdir("data")):
    file_path = os.path.join("data",file)
    print(file, pd.read_csv(file_path).shape)

ALS_ATAC_Astro.csv (17741, 1693)
ALS_ATAC_Exc.csv (19221, 4538)
ALS_RNA_Astro.csv (17941, 2901)
ALS_RNA_Exc.csv (17941, 3574)
Control_ATAC_Astro.csv (18836, 2017)
Control_ATAC_Exc.csv (7181, 4571)
Control_RNA_Astro.csv (17941, 3244)
Control_RNA_Exc.csv (17941, 2870)
